In [2]:
import numpy as np

# class that will do batching for the algorithm
# this code can bu reusable, just change the datas
class Audiobooks_Data_Reader():
    
    # batch_size is None by default
    def __init__(self, dataset, batch_size=None):
        
        npz = np.load('audiobooks_data{0}.npz'.format(dataset))
        self.inputs, self.targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.float)
        
        if batch_size is None:
            self.batch_size = self.inputs.shape[0]
        else:
            self.batch_size = batch_size
        self.curr_batch = 0
        self.batch_count = self.inputs.shape[0] // self.batch_size
        
    def __next__(self):
        if self.curr_batch >= self.batch_count:
            self.curr_batch = 0
            raise StopIteration()
            
        # batch_slice contains a range like (501,601)
        batch_slice = slice(self.curr_batch * self.batch_size, (self.curr_batch+1) * self.batch_size)

        inputs_batch = self.inputs[batch_slice]
        targets_batch = self.targets[batch_slice]
        self.curr_batch += 1
        
        classes_num = 2
        
        # targets_batch.shape[0]xclasses_num array full with 0s
        # number of rows x callses_num array full with 0s
        targets_one_hot = np.zeros((targets_batch.shape[0], classes_num))
        
        # range(targets_batch.shape[0]) -> specify our indices 
            #(1.row , 5.row or 10.row, ..., shape[0].row)
        # targets_batch contains our targets. Our targets are just 0s and 1s that they're in array
            # we start from the first number of the array targets_batch.
            # if this number is 0, so column number would be 0
                # if column is 0, our current idex would look like this -> [1,0]
            # if this number is 1, so column number would be 1
                # if column is 1, our current index would look like this -> [0,1]
            # cause we make it 1 for that specific row and column
            # column is choosen by if our number is 0 or 1
        # if we had more then 2 column we would just increase our classes_num
        targets_one_hot[range(targets_batch.shape[0]), targets_batch] = 1
        
        return inputs_batch, targets_one_hot
    
    def __iter__(self):
        return self

In [5]:
import tensorflow as tf

input_size = 10
output_size = 2
hidden_layer_size = 50

# clears the memory of all variables left from previous run
# reset the computational graph, rather then start from the beginning
tf.reset_default_graph()

inputs = tf.placeholder(tf.float32, [None, input_size])
targets = tf.placeholder(tf.int32, [None, output_size])

weights_1 = tf.get_variable("weights_1", [input_size, hidden_layer_size])
biases_1 = tf.get_variable("biases_1", [hidden_layer_size])

# nn is a module of tf and contains neural network support
# among other things, it contains the most commonly used activation functions
# tanh, relu, softmax, sigmoid
outputs_1 = tf.nn.relu(tf.matmul(inputs,weights_1) + biases_1)

weights_2 = tf.get_variable("weights_2", [hidden_layer_size, hidden_layer_size])
biases_2 = tf.get_variable("biases_2", [hidden_layer_size])

outputs_2 = tf.nn.relu(tf.matmul(outputs_1,weights_2) + biases_2)

weights_3 = tf.get_variable("weights_3", [hidden_layer_size, output_size])

# biases_3 -> Nonexoutput_size 
biases_3 = tf.get_variable("biases_3", [output_size])

# we use a trick at the output layer and don't use activation function as relu
outputs = tf.matmul(outputs_2, weights_3) + biases_3

# applies a softmax activation and calculates a cross-entropy loss
# this function scales our output_layer, even if it has a too small numbers, at the end they'll all be scaled
    # and problem will be solved
loss = tf.nn.sotmax_cross_entropy_with_logits(logits=outputs, labels=targets)

# finds the mean of the elements of a tensor across a dimensions
mean_loss = tf.reduce_mean(loss) 

# try to minimize mean_loss with optimizer
optimize = tf.train.AdamOptimizer(learning_rate=0.001).minimize(mean_loss)

# compare the all indices one by one and store the output as boolean 1 or 0 to the out_equals_target columns vector
# out_equals_target is a column vector that contains just 0s and 1s (not equal or equal)
out_equals_target = tf.equal(tf.argmax(outputs,1), tf.argmax(targets,1))

# tf.cast is a method that cast an object to another data type
accuracy = tf.reduce_mean(tf.cast(out_equals_target, tf.float32))

sess = tf.InteractiveSession()

initializer = tf.global_variables_initializer()

sess.run(initializer)

batch_size = 100

max_epochs = 50

prev_validation_loss = 9999999.

train_data = Audiobooks_Data_Reader('train', batch_size)

# we propagte forward just once in the validation and in the test set
validation_data = Audiobooks_Data_Reader('validation')

for epoch_counter in range(max_epochs):
    
    curr_epoch_loss = 0
    
    for input_batch, target_batch in train_data:
        _, batch_loss = sess.run([optimize, mean_loss],
                                 feed_dict={inputs:inputs_batch, targets:targets_batch})
        
        curr_epoch_loss += batch_loss
    
    curr_epoch_loss /= train_data.batch_count
    
    validation_loss = 0.
    validation_accuracy = 0.
    
    for input_batch, target_batch in validation_data:
        validation_loss, validation_accuracy = sess.run([mean_loss, accuracy],
                                 feed_dict={inputs:inputs_batch, targets:targets_batch})
        
    print('Epoch' + str(epoch_counter+1)+
          '.Training loss: '+'{0:.3f}'.format(curr_epoch_loss)+
          '. Validation loss: '+'{0:.3f}'.format(validation_loss)+
          '. Validation accuracy: '+'{0:.2f}'.format(validation_accuracy*100.)+'%')
    
    if validation_loss > prev_validation_loss:
        break
    
    prev_validation_loss = validation_loss

print('Validation loss is higher then the previous validation loss. Session has ended.')

AttributeError: module 'tensorflow' has no attribute 'placeholder'

In [ ]:
test_data = Audiobooks_Data_Reader('test')

for input_batch, target_batch in test_data:
       test_accuracy = sess.run([accuracy],
                                 feed_dict={inputs:inputs_batch, targets:targets_batch})
test_accuracy_percent = test_accuracy[0]*100.

print('Test accuracy: '+'{0:.2f}'.format(test_accuracy_percent)+'%')
